### Initialization

In [ ]:
# Importing libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import gc
from PIL import Image
import os
import random
import pickle

# Scikit-Learn Tools
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Tensorflow Tools
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Experimenting with Dask
import dask.array as da

In [ ]:
# Checking if Tensorflow is running on GPU
print(f"Built With CUDA: { tf.test.is_built_with_cuda() }")
print(f"Available GPU: { tf.config.list_physical_devices('GPU') }")

### Set Model IO Variables



#### Retrain Model?
* If the model should be trained from scratch, set retrainModel to **True**  
* If you want to import saved weights from a pretrained model set retrainModel to **False**

***

#### Override Weights?
* If newly trained weights should replace the old saved weights set overrideOldWeights to **True**  
* If you wish to keep the old weights saved as they are set overrideOldWeights to **False**

In [ ]:
# Setting model IO variables
retrainModel = True
pretrainedWeightsPath = ''

### Loading Image Data

In [ ]:
# Setting up empty lists for data
imageLabels, imageData = [], []

In [ ]:
# Function to fetch images and turn them into numpy arrays
def loadImageData(mainDirectory):
    # Iterating over categories
    for category in os.listdir(mainDirectory):
        # Iterating over images in category
        for imageName in tqdm(os.listdir(os.path.join(mainDirectory + category)), desc = f'Loading {category}'):
            # Loading image with pillow
            image = Image.open(os.path.join(mainDirectory, category, imageName))
            # Converting to numpy array
            data = np.asarray(image)

            # Appending label and data to training lists
            imageLabels.append(category)
            imageData.append(data)


In [ ]:
# Loading images
loadImageData('./Data/')

### Processing Image Data

In [ ]:
# Shuffling data and labels in unison
shuffleFrame = list(zip(imageData, imageLabels))
random.shuffle(shuffleFrame)

imageData, imageLabels = zip(*shuffleFrame)

In [ ]:
# Converting image data to numpy array
imageData = np.array(imageData)

In [ ]:
# One-Hot encoding label list
imageLabels = np.array(imageLabels).reshape(-1, 1)

encoder = OneHotEncoder()
imageLabels = encoder.fit_transform(imageLabels).toarray()

In [ ]:
# Reshaping data to fit model and augmentation generator
# Shape info: (Number of images, Height of image, Width of image, Channels)
# 'Number of images' is -1 so this dimension will be determined by numpy automatically based on the other fixed parameters
# 'Channels' is 1 because we use greyscale images -> only one color channel
imageData = imageData.reshape(-1, 240, 640, 1)

In [ ]:
# Assessing shape of training data
print(f"Data Shape: {imageData.shape}")
print(f"Labels Shape: {imageLabels.shape}")

In [ ]:
# Splitting into train and test data
trainData, testData, trainLabels, testLabels = train_test_split(imageData, imageLabels, test_size = 0.25, random_state = 111)

In [ ]:
# Splitting turning train data into Dask array
chunkedTrainData = da.from_array(trainData, chunks = (1250, 240, 640, 1)) # 1250
chunkedTrainLabels = da.from_array(trainLabels, chunks = (1250, 10))

# Splitting turning test data into Dask array
chunkedTestData = da.from_array(testData, chunks = (420, 240, 640, 1)) # 420
chunkedTestLabels = da.from_array(testLabels, chunks = (420, 10))

In [ ]:
gc.collect()

### Pickling Image Data

In [ ]:
# Saving processed data to pickle
dataPickle = open('Pickles/imageData.pkl', 'wb')
pickle.dump(imageData, dataPickle)
dataPickle.close()

# Saving processed labels to pickle
labelPickle = open('Pickles/imageLabel.pkl', 'wb')
pickle.dump(imageLabels, labelPickle)
labelPickle.close()

### Data Augmentation With ImageDataGenerators

In [ ]:
# Defining training data generator
trainDataGen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True
)

# Defining test data generator
testDataGen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True
)

### Defining Model

In [ ]:
# Creating model called locana (Sanskrit for 'Vision')
locana = Sequential()

In [ ]:
# Setting dummy input to avoid OOM error
dummy = tf.zeros((1, 240, 640, 1))
locana._set_inputs(dummy)

In [ ]:
# Setting up layers

# Convolutional layer 1
locana.add(Conv2D(32, kernel_size = 5, activation = 'relu', input_shape = (240, 640, 1)))
locana.add(MaxPooling2D((2, 2)))

# Convolutional layer 2
locana.add(Conv2D(64, kernel_size = 3, activation = 'relu'))
locana.add(MaxPooling2D((2, 2)))

# Convolutional layer 3
locana.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
locana.add(MaxPooling2D((2, 2)))

# Flattening and first dense layer
locana.add(Flatten())
locana.add(Dense(128, activation = 'relu'))

# Output layer
locana.add(Dense(10, activation = 'softmax'))

In [ ]:
# Compiling model
locana.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training Model

In [ ]:
# Setting training parameters
batchSize = 8
epochsPerChunk = 4

In [ ]:
# Function managing individual train iteration
def trainingIteration(trainChunk, trainChunkLabels, testChunk, testChunkLabels):
    # Fitting training data generator to train data
    trainDataGen.fit(trainChunk)
    
    # Fitting test data generator to test data
    testDataGen.fit(testChunk)
    
    # Training model on given chunk
    locana.fit(
        trainDataGen.flow(trainChunk, trainChunkLabels, batch_size = batchSize),
        validation_data = testDataGen.flow(testChunk, testChunkLabels, batch_size = batchSize),
        steps_per_epoch = len(trainChunk) / batchSize,
        epochs = epochsPerChunk
    )

In [ ]:
def trainModel():
    # Iterating over chunks in train data dask array
    for index, trainChunk in tqdm(enumerate(chunkedTrainData.blocks), total = 12, desc = "Training On Chunks"):
        # Retrieving appropriate train label chunk from Dask array
        trainLabels = chunkedTrainLabels.blocks[index]
        # Retrieving appropriate test data chunk from Dask array
        testChunk = chunkedTestData.blocks[index]
        # Retrieving appropriate test label chunk from Dask array
        testLabels = chunkedTestLabels.blocks[index]

        print(f'- - - Chunk {index + 1} - - -')
        # Executing train iteration with current chunk data
        trainingIteration(trainChunk, trainLabels, testChunk, testLabels)
        # Garbage collection to keep memory clean
        print(f'Collected Garbage: {gc.collect()}')

In [ ]:
if retrainModel:
    # Training model on train data
    trainModel()
    # Overriding old saved weights if so chosen
    if len(pretrainedWeightsPath) == 0: locana.save_weights('pretrainedLocana.h5')
else:
    # Load pretrained GPU weights
    locana.load_weights(pretrainedWeightsPath)